In [ ]:
'''

Looping through multiple tickers

Trying pycaret time series with daily S&P features (with beta and rf).
1200 rows
target = 'M1_forward_return'
fold_strategy = 'expanding'
fold=5
fh=21

Dropped Mkt_M1_forward_return

SUCCESSFUL Out of sample forecast
'''

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycaret
#from pycaret.regression import *
from pycaret.time_series import *

from pycaret.utils.time_series import clean_time_index

In [ ]:
#Tickers to loop over
Ticker_list=["NVDA","META","TSLA","AMZN","GOOG","MSFT","AAPL","GS","JPM","NFLX","NKE","AAL","LUV","GM"]

In [ ]:
# Dictionary to store DataFrames
dfs = {}

for ticker in Ticker_list:
    # Replace 'path/to/your/file.csv' with the actual path to your CSV file
    filepath = f'/Users/bennyang/Downloads/{ticker} features.csv'
    
    # Use pandas to read the CSV file
    dfs[ticker] = pd.read_csv(filepath)
    
    # Print the dataframe to verify it was read correctly
    #print(dfs)

In [ ]:
for i in dfs.values():
    i['Date'] = pd.to_datetime(i['Date'])

In [ ]:
# Replace 'path/to/your/file.xlsx' with the actual path to your Excel file
Beta_rf = '/Users/bennyang/Downloads/beta&rf.csv'

# Use pandas to read the Excel file
Beta_rf = pd.read_csv(Beta_rf)

# Print the dataframe to verify it was read correctly
print(Beta_rf)

In [ ]:
#Extract beta data and combine it with the main dataframe
for key,value in dfs.items():
    beta = Beta_rf[Beta_rf['Ticker'] == f'{key}']
    beta['Date'] = pd.to_datetime(beta['Date'])
    beta.drop('Ticker',axis=1,inplace=True)
    dfs[key]= pd.merge(dfs[key], beta,  how='left', left_on=['Date'], right_on = ['Date'])
    #Drop market 1 month forward return
    dfs[key].drop('Mkt_M1_forward_return',axis=1,inplace=True) 

In [ ]:
'''
#Index the dates and set frequency
for key,value in dfs.items():
    dfs[key].set_index("Date", inplace=True)
    dfs[key] = dfs[key].asfreq('B')

'''

In [ ]:
#Get the latest features to avoid NA
Latest_Features = {}
Test_Features = {}
M1_forward_return = {}

#Need to edit the last values as necessary to avoid NA
for key,value in dfs.items():
    Latest_Features[key]= dfs[key][-1400:-221]
    Test_Features[key]=dfs[key][-221:]
    M1_forward_return[key]=Test_Features[key]['M1_forward_return']
    Test_Features[key].drop('M1_forward_return',axis=1,inplace=True)

In [ ]:
Latest_Features['TSLA']['M1_forward_return']

In [ ]:
#Using top 3 models to forecast
for key,value in Latest_Features.items():
    
    model = setup(Latest_Features[key], target = 'M1_forward_return',fold_strategy = 'expanding', fold=5, fh=21, n_jobs=-1,
                  scale_target = 'zscore', scale_exogenous= 'zscore',
                  session_id=123, numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
                  log_experiment=False, experiment_name=f'{key} experiment')

    best_3model = compare_models(n_select = 3,exclude = "auto_arima")
    blend = blend_models(best_3model)
    # plot forecast for 21 sessions in future
    plot_model(blend, plot = 'forecast', data_kwargs = {'fh' : 21})

In [ ]:
#Using solely ARIMA to forecast
for key,value in Latest_Features.items():
    
    model = setup(Latest_Features[key], target = 'M1_forward_return',fold_strategy = 'expanding', fold=5, fh=21, n_jobs=-1,
                  scale_target = 'zscore', scale_exogenous= 'zscore',
                  session_id=123, numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
                  log_experiment=False, experiment_name=f'{key} experiment')

    #best_3model = compare_models(n_select = 3,exclude = "auto_arima")
    #blend = blend_models(best_3model)
    # plot forecast for 21 sessions in future
    arima = create_model('arima')
    plot_model(arima, plot = 'forecast', data_kwargs = {'fh' : 21})
    final_model = finalize_model(arima)
    test_length=Test_Features[key].shape[0]
    y_pred=predict_model(final_model , X = Test_Features[key],fh=test_length)
    
    plt.figure()
    plt.plot(y_pred,color='green',label=f'{key} Prediction')
    plt.plot(M1_forward_return[key],color='red',label=f'{key} Actual')
    plt.title(f'{key} Actual vs predicted 1 month ahead returns')
    plt.xlabel("Session")
    plt.ylabel("Return 1 month ahead")
    plt.legend()
    plt.show();